In [1]:
import pandas as pd
import numpy as np

import keras as K
import tensorflow as tf
import time

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
class Config(object):
    pass

config = Config()
config.log_dir = 'tf_logs/cnn/'+str(time.strftime('%Y-%m%d %H:%M:%S'))
config.batch_size = 100
config.epochs = 400
config.kernel_size = 3 # we will use 3x3 kernels throughout
config.pool_size = 2 # we will use 2x2 pooling throughout
config.conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
config.conv_depth_2 = 64 # ...switching to 64 after the first pooling layer

config.pool_strides = 2
config.conv_strides = 2

In [3]:
# fix random seed for reproducibility
np.random.seed(42)

In [4]:
from dance.audiosamples import RectifiedAudioSamples as AudioSamples

In [5]:
train_data = pd.read_pickle("train_data.pkl")
val_data = pd.read_pickle("val_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

In [6]:
def generate_samples(train_data, val_data, test_data, sampler):
    try:
        del(train_samples)
        del(val_samples)
        del(test_samples)
    except:
        pass
    train_samples = sampler(train_data, subsample_rate=2)
    scaler = train_samples.norm()
    encoder = train_samples.encode()
    val_samples=sampler(val_data, subsample_rate=2)
    val_samples.norm(scaler)
    val_samples.encode(encoder)
    test_samples = sampler(test_data, subsample_rate=2)
    test_samples.norm(scaler)
    test_samples.encode(encoder)
    return train_samples, val_samples, test_samples

In [7]:
train, val, test = generate_samples(train_data, val_data, test_data, AudioSamples)

In [8]:
tf.reset_default_graph()
model = K.models.Sequential()
model.add(K.layers.Conv1D(config.conv_depth_1, config.kernel_size, input_shape=(train.X.shape[1],1,), activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.MaxPooling1D(config.pool_size, config.pool_strides))
# model.add(K.layers.DropOut(config.drop_rate))
model.add(K.layers.Conv1D(config.conv_depth_2, config.kernel_size, activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.MaxPooling1D(config.pool_size, config.pool_strides))
# model.add(K.layers.BatchNormalization())
model.add(K.layers.Conv1D(config.conv_depth_2, config.kernel_size, activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.MaxPooling1D(config.pool_size, config.pool_strides))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(train.Y.shape[-1], activation='softmax', kernel_regularizer=K.regularizers.l2()))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
tbCallback = K.callbacks.TensorBoard(config.log_dir, batch_size=config.batch_size, histogram_freq=10)
patience = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

history = model.fit(train.get_as_timeseries(), train.Y, epochs=config.epochs, 
                    batch_size=config.batch_size, 
                    validation_data=(val.get_as_timeseries(), val.Y), 
                    callbacks=[tbCallback, patience],
                    verbose=True,
                   )

Train on 39648 samples, validate on 10166 samples
INFO:tensorflow:Summary name conv1d_1/kernel:0 is illegal; using conv1d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_1/bias:0 is illegal; using conv1d_1/bias_0 instead.
INFO:tensorflow:Summary name conv1d_2/kernel:0 is illegal; using conv1d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_2/bias:0 is illegal; using conv1d_2/bias_0 instead.
INFO:tensorflow:Summary name conv1d_3/kernel:0 is illegal; using conv1d_3/kernel_0 instead.
INFO:tensorflow:Summary name conv1d_3/bias:0 is illegal; using conv1d_3/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
Epoch 1/400
39648/39648 [==============================] - 599s - loss: 3.1333 - acc: 0.2756 - val_loss: 1.9775 - val_acc: 0.2310
Epoch 2/400
39648/39648 [==============================] - 6037s - loss: 1.8691 - acc: 0.2924 - val_loss

In [10]:
model.evaluate(test.get_as_timeseries(), y=test.Y, verbose=True)

18384/18384 [==============================] - 73s    


[2.5811073852685142, 0.13163620539599652]